In [2]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
CALCULATED_CARD_DRAW_DOWNS = dataiku.Dataset("CALCULATED_CARD_DRAW_DOWNS")
CALCULATED_CARD_DRAW_DOWNS_df = CALCULATED_CARD_DRAW_DOWNS.get_dataframe()

CALCULATED_CARD_DRAW_UPS = dataiku.Dataset("CALCULATED_CARD_DRAW_UPS")
CALCULATED_CARD_DRAW_UPS_df = CALCULATED_CARD_DRAW_UPS.get_dataframe()

In [8]:
df_down = CALCULATED_CARD_DRAW_DOWNS_df
df_down['DRAW_DOWN_DATE'] = pd.to_datetime(df_down['DRAW_DOWN_DATE']).dt.date
df_down.sort_values(['CUSTOMER'], inplace=True)
df_down.head()

,CUSTOMER,DRAW_DOWN_DATE,MEAN_DD,STD_DD,ACTIVE_CARD_MAX
0,3LUM,2019-12-01,13.000000,0.577350,14.0
193,3US6 (EQUIPMENT),2021-08-01,36.600000,27.673092,104.0
192,ACORDA THERAPEUTICS,2019-09-01,129.600000,2.190890,133.0
191,ADF PIZZA MANAGEMENT,2020-12-01,7.500000,1.623688,12.0
190,ALENT,2020-04-01,1.333333,0.492366,2.0


In [9]:
df_up = CALCULATED_CARD_DRAW_UPS_df
df_up['DRAW_UP_DATE'] = pd.to_datetime(df_up['DRAW_UP_DATE']).dt.date
df_up.sort_values(['CUSTOMER'], inplace=True)
df_up.head()

,CUSTOMER,DRAW_UP_DATE,MEAN_DU,STD_DU,ACTIVE_CARD_MAX
0,3GPP,2020-08-01,27.166667,4.239068,35.0
321,3LUM,2019-01-01,12.111111,1.615893,14.0
320,3M (3MUS),2020-04-01,2375.083333,169.146767,2728.0
319,3M COMPANY (3MVR),2020-08-01,360.083333,65.468186,399.0
318,3M COMPANY3MUS,2021-08-01,670.111111,196.492013,963.0


In [0]:
#POTENTIAL_DUPLICATES_df = ... # Compute a Pandas dataframe to write into POTENTIAL_DUPLICATES

# Write recipe outputs
#POTENTIAL_DUPLICATES = dataiku.Dataset("POTENTIAL_DUPLICATES")
#POTENTIAL_DUPLICATES.write_with_schema(POTENTIAL_DUPLICATES_df)